# The method to load circuits in `QASM` format.
?

# Explanation to the Optimization Techniques
## Pauli String Grouping Technique
### Abstract
One of the main methods proposed to suppress an increase in the number of measurements is the partitioning method, where Pauli strings are partitioned so that their expectation values can be measured simultaneously. 

In this method, a group of Pauli strings is divided into subgroups, and all the components of each subgroup are measured simultaneously using only one circuit (hereafter, such subgroups are called “partitions”). 

### Implementation
The problem of Pauli string grouping can be broken down into two parts:
1. The partition of a given group of Pauli strings.
2. The method to get the equivalent measurement results of pauli strings in a partition from the result of the so-called "simultaneous measurement" 

We solve the first part (the partition) of the problem by (?xxx L) calling the function/method in the `Estimator`, and we tackle the second part by using the function xxx in xxx.py

this is the workflow of xxx:
1. xxx



## Error Mitigation Technique


### Readout Error Mitigation
#### Abstract
Readout-Error Mitigation (REM) is an error mitigation technique in which inverted transition / confusion matrices are applied to the noisy measurement results.

technique is based on two main ideas:

1. Generating a confusion matrix for a specific device;

2. Computing the psuedoinverse of this confusion matrix and applying it to the raw measurement (or “readout”) results.

#### Implementation
##### Measurement based method
The readout mitigator is generated from an assignment matrix: a $2^n \times 2^n$ matrix A such that $A_{y,x}$ is the probability to observe y given the true outcome should be x. The assignment matrix is used to compute the mitigation matrix used in the readout error mitigation process itself.

A Correlated readout mitigator uses the full $2^n \times 2^n$ assignment matrix, meaning it can only be used for small values of n. 

We use correlated readout mitigation in our program for better accuracy.

we can get the mitigator(derived from assignment matrix) by the following measurement and process procedure:

```python
exp = LocalReadoutError(qubits)
for c in exp.circuits():
    print(c)
result = exp.run(backend)
mitigator = result.analysis_results(0).value
```
After we get the mitigator, it's easy to calculate the mitigated expectation value of a diagonal observable (namely a Pauli string) from the measurement result:
`expectation=mitigator.expectation_value(data=measurement_count, diagonal=pauli,  shots=shots)`


##### Noise model based method
the figure below shows how to use REM by mitiq.rem.
![Alt text](image.png)

Since the estimated confusion matrix A is circuit-independent—it characterizes the readout noise of the device regardless of what circuit is being executed, we can get matrix A directly from the noise model.

We can obtain the mitigator by: `mitigator = qiskit.result.CorrelatedReadoutMitigator(assignment_matrices=matrix_A)`

### Zero-Noise Extrapolation (ZNE)
The figure below shows the main procedure of ZNE mitigation method:
![Alt text](image-1.png)

#### Abstract
Zero noise extrapolation (ZNE) is an error
mitigation technique used to extrapolate the noiseless expectation value of an
observable from a range of expectation values computed at different noise levels.
This process works in two steps:

- **Step 1: Intentionally scale noise**. This can be done with different methods.
*Pulse-stretching*  can be used to increase the noise level of a quantum computation. Similar results can be obtained, at a gate-level, with *unitary folding* or *identity insertion scaling*.

- **Step 2: Extrapolate to the noiseless limit**. This can be done by
fitting a curve (often called *extrapolation model*) to the expectation values measured at different noise levels to extrapolate the noiseless expectation value.

#### Implementation
##### First Step: Generating noise-scaled quantum circuits
  - We provide a `QPROGRAM`, i.e., a `QuantumCircuit` defined by Qiskit.
  - Mitiq generates a set of noise-scaled circuits by applying a scaling method (*unitary folding* or *identity insertion scaling*) with different scale factors.
  - The noise-scaled circuits are executed on the noisy backend obtaining a set of noise-scaled expectation values.

##### Second Step: Inferring the zero-noise value from the measured results
  - A parametrized curve is fit to the noise-scaled expectation values obtained in step one.
  - The curve is extrapolated to the zero-noise limit, obtaining an error mitigated expectation value.

##### The optimization of parameters for ZNE
There are three key parameters in ZNE that can affect its performance: 
- the scale factors, i.e., [1.0, 2.0, 3.0] (which is the optimal choice, from our previous experiment)
- the folding method (which inserts gates into original circuit to scale the noise), i.e., the fold_global method (which, from our previous experiment, is also the optimal choice)
- the extrapolation method, i.e., Rechardson extrapolation method (which is the optimal choice, from our previous experiment)



## Transpile Optimization
We flexibly make use of the transpile optimization to improve the performance of our program. 
The function we called to optimize the circuit transpiling is xxx.transpile(optimization_level=xxx)
here we set the otimization_level to 3 in certain parts of our procedure, to mitigate the extent of the circuit decoherence and quantum gate noise.
and in some other parts of the process, we set optimization_level to 0 with the aim of avoiding disturbing the noise scaled quantum circuit, otherwise the inserted gates pair like $X$ and $X^{\dagger}$ which is equivalent to an identity gate in theory but invokes noise in practice.


# Alignment with the qiskit estimator
